In [40]:
#import libraries
import pandas as pd
import re

In this notebook I'm working to standardize data downloads. Inputs to this notebook are the data in whatever form they can be downloaded as (these raw files are kept in the `raw-data` directory. Output of this notebook are tsv files in the form of `country \t quantity`. Names are outputted in their canonical snakecase form, but non-Americas countries are _not_ trimmed out at this point.

Raw data files that need standardization are:

* `americas-zika-suitability.tsv` (Messina et al Zika environmental suitability scores output from Mathematica notebook)
* `Americas-to-Americas-2012-2016_KK_DB.xlsx` (Datafile made available from Kamran Kahn)
* `pop-weighted-centroids.tsv` (Population weighted centroids for each country, output from Mathematica)
* `proportion-urban-population-worldbank.csv` (Proportion of population living in an urban area, direct download from WorldBank)
* `raw-cia-world-factbook-pop-sizes.txt` (Country population size, direct download from CIA world factbook)

In [21]:
# define naming conversion function
def camelcase_to_snakecase(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower().replace(' ','').replace('-','').replace(',_the','').replace('andthe_','')

In [41]:
#CIA world factbook download is particularly bad. Commas, weird white space, and other annoying formatting, so dealing with this
#one on its own first.
raw_popsize_file = '/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/raw-data/raw-cia-world-factbook-pop-sizes.txt'
popsize_dict = {}
with open(raw_popsize_file,'r') as file:
    for line in file:
        line = line.replace(',','').replace(' ','')
        country = re.findall('\d+|\D+', line)[1]
        snakecase_country = camelcase_to_snakecase(country)
        pop_size = re.findall('\d+|\D+', line)[2]
        popsize_dict[snakecase_country] = pop_size

Looks like this fixes the whitespace issues, and fixes almost all the names, with the exception of the `virgin_islands` which should be `united_states_virgin_islands`. I'll fix that below using my name fix tsv file.

In [42]:
#load in raw datasets as dataframes
#at this point only the popsize_df has the names in the correct format, others are fixed below.
popsize_df = pd.DataFrame(list(popsize_dict.iteritems()), columns=['country','population_size'])

pop_weighted_centroids_df = pd.read_csv('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/raw-data/pop-weighted-centroids.tsv', sep='\t', names=['country','latitude','longitude'])

suitability_df = pd.read_csv('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/raw-data/americas-zika-suitability.tsv', sep='\t',names=['Country','suitability_score'])

pax_traffic_df = pd.read_excel('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/raw-data/Americas-to-Americas-2012-2016_KK_DB.xlsx')
pax_traffic_df.columns = ['pax_volume','originCamelCase','destinationCamelCase','year']

urban_pop_df = pd.read_csv('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/raw-data/proportion-urban-population-worldbank.csv', sep='\t')

In [45]:
##### Standardize Country Naming for Suitability, Passenger Air (PAX traffic) and Urban proportion of population dataframes #####

#Switch names to snake_case with the above function. This will fix most country names, but not all.
pax_traffic_df['origin'] = pax_traffic_df['originCamelCase'].apply(camelcase_to_snakecase)
pax_traffic_df['destination'] = pax_traffic_df['destinationCamelCase'].apply(camelcase_to_snakecase)
urban_pop_df['Country Name'] = urban_pop_df['Country Name'].apply(camelcase_to_snakecase)
suitability_df['Country'] = suitability_df['Country'].apply(camelcase_to_snakecase)
pop_weighted_centroids_df['country'] = pop_weighted_centroids_df['country'].apply(camelcase_to_snakecase)
#To switch the remaining, more problematic names, I'm going to use a name fix tsv file.

#read in name fix tsv, making a dictionary where the key is the name in it's messed up form, and the value 
#is the standardized name in the form I want.
name_fix_df = pd.read_table('/Users/alliblk/Desktop/gitrepos/zika-usvi/scripts/name_fix.tsv',sep='\t')
name_fix_dict = dict(name_fix_df.values)
# make a new column to hold the standardized names
# note that I'm not fixing any messed up names of countries that are not part of our analysis.
# eg bosniaand_herzegovina should be left as is since it's not part of the analysis.
# therefore if name_fix_dict.get() is None I want the same (messed up) row name to be returned.
# all of this will be saved as a new series and appended to the df.
# note axis = 1 just says loop through the rows.

popsize_df['country'] = popsize_df.apply(lambda row: name_fix_dict.get(row['country'],row['country']), axis=1)
pop_weighted_centroids_df['country'] = pop_weighted_centroids_df.apply(lambda row: name_fix_dict.get(row['country'],row['country']), axis=1)
urban_pop_df['country'] = urban_pop_df.apply(lambda row: name_fix_dict.get(row['Country Name'],row['Country Name']),axis=1)
pax_traffic_df['origin'] = pax_traffic_df.apply(lambda row: name_fix_dict.get(row['origin'],row['origin']),axis=1)
pax_traffic_df['destination'] = pax_traffic_df.apply(lambda row: name_fix_dict.get(row['destination'],row['destination']),axis=1)
suitability_df['country'] = suitability_df.apply(lambda row: name_fix_dict.get(row['Country'],row['Country']), axis=1)

##### for Urban Population, turn percent (0 to 100 scale) to fraction (0 to 1 scale). Using the 2016 urban proportion data.
urban_pop_df['2016_urban_pop_fraction'] = urban_pop_df['2016']/100.0

In [47]:
#Alright, so we've fixed up the raw data frames nicely. Export to tsv.
pop_weighted_centroids_df.to_csv('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/standardized-data/std-population-weighted-centroids.tsv', index = False, columns = ['country', 'latitude','longitude'], sep = '\t')
popsize_df.to_csv('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/standardized-data/std-population-size.tsv', index = False, columns = ['country', 'population_size'], sep = '\t')
suitability_df.to_csv('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/standardized-data/std-zika-suitability.tsv', index = False, columns = ['country', 'suitability_score'], sep = '\t')
pax_traffic_df.to_csv('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/standardized-data/std-pax-volume.tsv', index=False, columns = ['origin', 'destination' , 'pax_volume'], sep = '\t')
urban_pop_df.to_csv('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/standardized-data/std-urban-pop-fraction.tsv', index=False, columns = ['country','2016_urban_pop_fraction'], sep='\t')